In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import missingno as msno
%matplotlib inline

In [3]:
log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id', 'time', 'bet', 'win']

users = pd.read_csv('users.csv', encoding='KOI8-R', sep='\t')
users.columns = ['user_id','email','geo']
users.user_id = users.user_id.apply(lambda x:x.lower())

log = log[log.user_id != '#error']
log.user_id = log.user_id.str.split('- ').apply(lambda x: x[1])
log.bet = log.bet.fillna(0)

def fillna_win(row):
    if row.win > 0:
        return row.win
    elif row.bet > 0:
        return -row.bet
    return 0

log['win'] = log.apply(lambda row:fillna_win(row),axis=1)
log['time'] = pd.to_datetime(log['time'].apply(lambda x: x[1:]))

def net_(row):
    if row.win < 0:
        return row.win
    return (row.win-row.bet)

log['net'] = log.apply(lambda row:net_(row),axis=1)

df = pd.merge(log, users, on="user_id")
df

,user_id,time,bet,win,net,email,geo
0,user_919,2019-01-01 14:06:51,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
1,user_919,2019-01-30 10:06:00,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
2,user_919,2019-02-05 14:33:44,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
3,user_919,2019-02-14 11:38:05,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
4,user_919,2019-03-02 04:23:36,300.0,-300.0,-300.0,Chikkaverle@icloud.com,Хабаровск
...,...,...,...,...,...,...,...
970,user_932,2019-02-24 22:40:06,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
971,user_932,2019-03-15 10:56:14,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
972,user_932,2019-03-18 10:13:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
973,user_932,2019-03-27 12:18:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск


In [4]:
df.groupby('user_id').net.sum().median()

1986.0

In [11]:
df.groupby('user_id').bet.sum().sort_values()

user_id
user_901        600.0
user_904        600.0
user_932        700.0
user_936        800.0
user_991        800.0
              ...    
user_987      46128.0
user_905      53395.0
user_910     232121.0
user_917     304221.0
user_942    1791528.0
Name: bet, Length: 100, dtype: float64

In [83]:
df[df.bet==0].groupby('user_id').bet.value_counts().mean()

5.05

In [34]:
df[df.user_id == 'user_942']

,user_id,time,bet,win,net,email,geo
193,user_942,2019-01-03 14:03:21,98753.0,-98753.0,-98753.0,Mailitry@ya.ru,Москва
194,user_942,2019-01-04 13:59:42,1678321.0,9876543.0,8198222.0,Mailitry@ya.ru,Москва
195,user_942,2019-01-05 10:44:56,0.0,0.0,0.0,Mailitry@ya.ru,Москва
196,user_942,2019-01-15 02:01:38,0.0,0.0,0.0,Mailitry@ya.ru,Москва
197,user_942,2019-01-24 05:03:17,0.0,0.0,0.0,Mailitry@ya.ru,Москва
198,user_942,2019-02-15 08:27:35,0.0,0.0,0.0,Mailitry@ya.ru,Москва
199,user_942,2019-02-26 22:10:18,600.0,-600.0,-600.0,Mailitry@ya.ru,Москва
200,user_942,2019-03-09 19:57:04,800.0,-800.0,-800.0,Mailitry@ya.ru,Москва
201,user_942,2019-03-18 04:21:14,800.0,-800.0,-800.0,Mailitry@ya.ru,Москва
202,user_942,2019-03-29 07:40:02,1000.0,7384.0,6384.0,Mailitry@ya.ru,Москва


In [59]:
t_0 = pd.DataFrame(df[df.bet==0].groupby('user_id').time.min()).reset_index()
t_more = pd.DataFrame(df[df.bet>0].groupby('user_id').time.min()).reset_index()
first_bet = pd.merge(t_0, t_more, on='user_id')
first_bet

,user_id,time_x,time_y
0,user_1000,2019-01-21 19:31:34,2019-01-28 20:57:37
1,user_900,2019-01-01 04:37:25,2019-02-08 10:36:47
2,user_901,2019-01-07 08:45:59,2019-03-14 13:51:20
3,user_902,2019-01-28 03:12:06,2019-03-09 16:52:11
4,user_903,2019-01-01 16:31:16,2019-02-08 11:06:00
...,...,...,...
95,user_995,2019-01-07 09:56:20,2019-02-26 15:29:25
96,user_996,2019-03-03 11:43:38,2019-01-10 07:32:17
97,user_997,2019-01-05 20:55:29,2019-03-05 11:05:39
98,user_998,2019-01-03 02:48:25,2019-01-25 08:57:20


In [63]:
def first_(row):
    if row.time_y > row.time_x:
        return row.time_y-row.time_x
    return row.time_x-row.time_x

first_bet['delta'] = first_bet.apply(lambda row:first_(row), axis=1)
first_bet

,user_id,time_x,time_y,delta
0,user_1000,2019-01-21 19:31:34,2019-01-28 20:57:37,7 days 01:26:03
1,user_900,2019-01-01 04:37:25,2019-02-08 10:36:47,38 days 05:59:22
2,user_901,2019-01-07 08:45:59,2019-03-14 13:51:20,66 days 05:05:21
3,user_902,2019-01-28 03:12:06,2019-03-09 16:52:11,40 days 13:40:05
4,user_903,2019-01-01 16:31:16,2019-02-08 11:06:00,37 days 18:34:44
...,...,...,...,...
95,user_995,2019-01-07 09:56:20,2019-02-26 15:29:25,50 days 05:33:05
96,user_996,2019-03-03 11:43:38,2019-01-10 07:32:17,0 days 00:00:00
97,user_997,2019-01-05 20:55:29,2019-03-05 11:05:39,58 days 14:10:10
98,user_998,2019-01-03 02:48:25,2019-01-25 08:57:20,22 days 06:08:55


In [64]:
first_bet['delta'].mean()

Timedelta('47 days 01:34:33.930000')

In [65]:
df.groupby('geo').win.sum().sort_values()

geo
Тюмень                 -499.0
Ставрополь              487.0
Краснодар              3248.0
Пермь                 19518.0
Арзангелтск           25813.0
Красноярск            27397.0
Екатеринбург          28582.0
Хабаровск             33451.0
Ижевск                34479.0
Ярославль             46845.0
Казань                75352.0
Санкт-Петербург      105345.0
Воронеж              107514.0
Москва             11379755.0
Name: win, dtype: float64

In [71]:
df[df.bet>0].groupby('geo').bet.mean()

geo
Арзангелтск         1415.911111
Воронеж             2119.745455
Екатеринбург         486.956522
Ижевск              2058.437500
Казань              1572.066667
Краснодар           2124.235294
Красноярск          3332.416667
Москва             62240.846154
Пермь               2498.800000
Санкт-Петербург     1447.500000
Ставрополь          3165.000000
Тюмень               560.000000
Хабаровск           1515.400000
Ярославль           1628.048780
Name: bet, dtype: float64

In [72]:
df[df.bet>0].groupby('geo').bet.mean().max()/df[df.bet>0].groupby('geo').bet.mean().min()

127.81602335164835

In [74]:
sample2 = df.groupby('geo').user_id.count()
sample2

geo
Арзангелтск         96
Воронеж             88
Екатеринбург        49
Ижевск              61
Казань              66
Краснодар           86
Красноярск          56
Москва              61
Пермь               55
Санкт-Петербург    115
Ставрополь          36
Тюмень              32
Хабаровск           60
Ярославль           89
Name: user_id, dtype: int64

In [77]:
log['time'].apply(lambda pandas_dataframe: pandas_dataframe.month) 

0      1
1      1
2      1
3      1
4      1
      ..
991    4
992    4
993    4
994    4
995    4
Name: time, Length: 985, dtype: int64

In [78]:
log.time.dt.month 

0      1
1      1
2      1
3      1
4      1
      ..
991    4
992    4
993    4
994    4
995    4
Name: time, Length: 985, dtype: int64